In [1]:
# load libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans

import warnings
warnings.simplefilter(action="ignore", category=Warning)

In [2]:
# load data
df = pd.read_csv("Article_top_Data.csv")
# # group by Date
# df['Day'] = df['Date'].map(lambda x: 1 if len(pd.bdate_range(x, x)) else 0 )

# # group by Hour
# df['Hour'] = df['Hour'].map(lambda x: 0 if int(x)>=5 and int(x)<=8
#                                             else 1 if int(x)>=9 and int(x)<=12
#                                             else 2 if int(x)>=13 and int(x)<=16
#                                             else 3)

# # group month by seasons
# df['Month'] = df['Month'].map(lambda x: 0 if int(x)>=3 and int(x)<=5 
#                                             else 1 if int(x)>=6 and int(x)<=8 
#                                             else 2 if int(x)>=9 and int(x)<=11
#                                             else 3)

# # add temperature for each seasons
# df.loc[(df.Month==0), 'Temp'] = 2.28
# df.loc[(df.Month==1), 'Temp'] = 8.9
# df.loc[(df.Month==2), 'Temp'] = 19.57
# df.loc[(df.Month==3), 'Temp'] = 10.06
df.head()

,Unnamed: 0,Date,StoreNumber,CashRegisterNumber,ReceiptNumber,ReceiptRowNumber,Time,ArticleNumber,ArticleName,Quantity,Timestamp,Month,Day,Hour,Group
0,4,2022-01-03,1001,1,296,5,09:21:14,419727,"GL H-Milch 3,5% 1L VLOG",1.0,2022-01-03 09:21:14,1,0,9,Milch
1,6,2022-01-03,1001,1,296,7,09:21:14,999999998025,Pfandartikel (Tandem) - EW,1.0,2022-01-03 09:21:14,1,0,9,Pfandartikel
2,7,2022-01-03,1001,1,296,8,09:21:14,435066,Schlaufentragetasche ST,1.0,2022-01-03 09:21:14,1,0,9,Schlaufentragetasche ST
3,19,2022-01-03,1001,1,300,1,09:25:34,421791,BO-Laugenbreze 1ST,1.0,2022-01-03 09:25:34,1,0,9,BO-Laugenbreze 1ST
4,20,2022-01-03,1001,1,300,2,09:25:34,421791,BO-Laugenbreze 1ST,1.0,2022-01-03 09:25:34,1,0,9,BO-Laugenbreze 1ST


In [3]:
store_sales = df.groupby(['StoreNumber','Group'], as_index=False).aggregate({'Quantity': 'sum'})
store_sales.head()

,StoreNumber,Group,Quantity
0,1001,"AugustinerHellMW1x0,5LFL",11256.000
1,1001,Avocado vorgereift ST,4576.000
2,1001,BO-Laugenbreze 1ST,18573.000
3,1001,Bananen Lose,13086.072
4,1001,Broetchen,83068.000


In [4]:
# Feature matrix with shape (n_series x time_observations)
series_df = store_sales.pivot(index='StoreNumber', columns='Group', values='Quantity')
series_df = series_df.dropna(axis='columns')
series_df

Group,Avocado vorgereift ST,Bananen Lose,Broetchen,GL Sahne 30% 200g VLOG,Gurken ST,Kiwi Gruen ST,M.I. Mozzarella oGt 125g,Milch,Pfandartikel,Schlaufentragetasche ST,VL Eier BH 10ST,Zeitschriften 7%
StoreNumber,,,,,,,,,,,,
1001,4576.0,13086.072,83068.0,6841.0,16291.0,6442.0,5863.0,19712.0,238341.0,9197.0,10827.0,8481.0
1007,7519.0,8566.780,38.0,6792.0,13315.0,5202.0,8159.0,17109.0,88155.0,5143.0,3558.0,9307.0
1028,6079.0,15234.682,73093.0,11418.0,20638.0,9355.0,7414.0,34897.0,283450.0,8477.0,10881.0,9533.0
1033,8140.0,11183.354,78174.0,9375.0,18340.0,10134.0,7601.0,22390.0,213086.0,8141.0,5374.0,1.0
1053,7746.0,6326.466,51913.0,7107.0,14934.0,7628.0,7171.0,16802.0,214036.0,8564.0,4653.0,6942.0
1098,5663.0,8483.850,53565.0,6575.0,16364.0,7609.0,6039.0,23366.0,185568.0,8244.0,6442.0,6774.0
1223,20458.0,23223.366,86947.0,10458.0,32750.0,15306.0,14488.0,22193.0,447970.0,21513.0,11208.0,15409.0
1303,7556.0,8813.946,40.0,8118.0,15463.0,6923.0,6471.0,16700.0,182892.0,8260.0,4830.0,13877.0
1311,7051.0,13949.484,65884.0,11907.0,20319.0,11092.0,7396.0,30644.0,321057.0,6347.0,9228.0,14041.0


In [5]:
feature_matrix = series_df.to_numpy()
print(feature_matrix[:3])

[[4.5760000e+03 1.3086072e+04 8.3068000e+04 6.8410000e+03 1.6291000e+04
  6.4420000e+03 5.8630000e+03 1.9712000e+04 2.3834100e+05 9.1970000e+03
  1.0827000e+04 8.4810000e+03]
 [7.5190000e+03 8.5667800e+03 3.8000000e+01 6.7920000e+03 1.3315000e+04
  5.2020000e+03 8.1590000e+03 1.7109000e+04 8.8155000e+04 5.1430000e+03
  3.5580000e+03 9.3070000e+03]
 [6.0790000e+03 1.5234682e+04 7.3093000e+04 1.1418000e+04 2.0638000e+04
  9.3550000e+03 7.4140000e+03 3.4897000e+04 2.8345000e+05 8.4770000e+03
  1.0881000e+04 9.5330000e+03]]


In [6]:
kmeans = KMeans(n_clusters=3)
kmeans.fit(feature_matrix)
labels = kmeans.labels_
print(labels)

[2 0 2 2 2 0 1 0 2 2 0 2 2 2 2]


In [7]:
# output every clusters
clusters = {}
for i in range(3):
    ind = np.where(labels==i)
    clusters[i]=series_df.index[ind].to_list()
print(clusters)

{0: [1007, 1098, 1303, 1436], 1: [1223], 2: [1001, 1028, 1033, 1053, 1311, 1417, 1487, 1504, 2801, 2808]}


In [8]:
from sklearn.metrics import silhouette_score

# 假设你有特征矩阵 feature_matrix 和聚类结果 labels

# 计算轮廓系数
silhouette_avg = silhouette_score(feature_matrix, labels)

print("聚类结果的轮廓系数: {:.2f}".format(silhouette_avg))


聚类结果的轮廓系数: 0.45
